# Scikit-Learn cross_val_score

Urmează partea în care o să ne uităm peste metoda cross_val_score din Scikit-Learn ce ne permite să realizăm Cross Validation pentru orice model. Utilizând asta nu mai trebuie să ne facem griji dacă modelul respectiv are implementat sau nu partea de Cross Validation (precum are implementat Ridge și Lasso care au versiunea de CV - ce provine de la Cross Validation). Pentru început o să revizuim partea de Cross Validation (K-Fold cross-validation)

Știm că pentru un set de date îl împărțim în set de date de train și test. Până în secțiunea aceasta am utilizat setul de testare și pentru partea de evaluare a performanței modelului, abia acuma am început să reținem o porțiune din setul de date pentru partea de evaluare a performanței (validation set). În ceea ce părivește cross-validation, prima dată este împărțit setul de date din nou în train-test set. Setul de testare este pus deoparte și nu trebuie să ne atingem de el. Cu ceea ce rămâne, adică setul de antrenare, trebuie să alegem o valoare pentru acel parametru de K-fold (în metoda de 'cross_val_score' poartă denumirea de cv). Acest parametru o să împartă setul de antrenare în atâtea părți câte îi oferim ca și valoare pentru acel parametru K-fold. Dacă îi oferim valorea 5 de exemmplu, atunci setul de antrenare o să fie împărțit în 5 seturi separate. Se face antrenarea folosind primele 4 seturi, iar validarea se va face pe ultimul set, după se va face antrenarea pe primele 3 și ultimul set, iar setul 4 va fi utilizat pentru validare. Se va repeta acest procedeu până ce se face validarea pentru fiecare set în parte (din cele 5).

Metoda cross_val_score din Scikit-Learn are nevoie de un model și un set de date de antrenare, împreună cu parametrul de K-fold (care poartă denumirea de cv). Acest procedeu funcționează cu orice model din Scikit-Learn și se realizează extrem de simplu și rapid.

<img src='../SS/cross_validation.png'>

Imaginea de mai sus reprezintă partea de cross-validation. Cu verde sunt prezentate seturile de date pe care se antrenează, iar cu albastru sunt seturile de date de validare. Se poate observa că utilizând partea de cross validation, modelul este antrenat și validat pe fiecare set de date. Din moment ce setul de date a fost împărțit în 5 (K-fold=5) o să se realizeze partea de antrenare a modelului de 5 ori. În funcție de setul de date cu care se lucrează trebuie să alegem numărul de k-folds. Dacă avem un set mare de date și alegem o valoare mare pentru k-fold, atunci o să dureze extrem de mult partea aceasta de cross_validation. Eroarea de verificare a modelului o să fie media celor 5 erori. În funcție de aceasta se modifică hyperparametrii modelului și la final se testează pe setul de date de testare

Să realizăm partea de cross-validation în Python și Scikit-Learn

In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read the data into a DataFrame
df = pd.read_csv('../data/DATA/Advertising.csv')

In [3]:
# print the head of the data
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [5]:
# Split the data into features and labels
X = df.drop('sales', axis=1)
y = df['sales']

In [6]:
# import the train_test_split method from Scikit-Learn
from sklearn.model_selection import train_test_split

In [7]:
# split the data into train-test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

Înainte de a crea un model trebuie din nou să ne importăm un scaler și să realizăm partea de scale data

In [8]:
# importing the scaler from sklearn
from sklearn.preprocessing import StandardScaler 

In [9]:
# creating an instance of the scaler
scaler = StandardScaler()

In [10]:
# fitting the scaler to the train data
scaler.fit(X_train)

StandardScaler()

In [11]:
# transforming the data with the scaler
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
# importing the Ridge model from Scikit-Learn
from sklearn.linear_model import Ridge

In [13]:
# creating an instance of the model
model = Ridge(alpha=100)

Până în acest moment, totul este identic cu ce s-a făcut anterior. Am luat datele, le-am transformat (cu un StandardScaler) după care am creat un model. De aici însă lucrurile încep să fie diferite. Trebuie să importăm metoda cross_val_score din Scikit-Learn pentru început. Aceasta se găsește în submodulul model_selection

In [14]:
from sklearn.model_selection import cross_val_score

In [15]:
help(cross_val_score)

Help on function cross_val_score in module sklearn.model_selection._validation:

cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)
    Evaluate a score by cross-validation.
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    estimator : estimator object implementing 'fit'
        The object to use to fit the data.
    
    X : array-like of shape (n_samples, n_features)
        The data to fit. Can be for example a list, or an array.
    
    y : array-like of shape (n_samples,) or (n_samples, n_outputs),             default=None
        The target variable to try to predict in the case of
        supervised learning.
    
    groups : array-like of shape (n_samples,), default=None
        Group labels for the samples used while splitting the dataset into
        train/test set. Only used in conjunction with a "Group" :term:`cv

Se poate observa că metoda cross_val_score() are un număr de parametrii importanți, și anume, estimator, X, y, cv și scoring. Estimator se referă la modelul pe care îl folosim pentru a realiza cross-validation (un model este doar un estimator, el estimează anumite valori). Prin urmare aici trebuie să oferim ca și valoare modelul creat mai sus (estimator=Ridge, sau se poate oferi doar simplu Ridge). X și y sunt datele de antrenare, features și labels. cv reprezintă valoarea pentru K-folds, în câte seturi să împartă datele de antrenare. Dacă nu se oferă nicio valoare, atunci o să meargă pe opțiunea de leave-one-out prin care se validează modelul cu o singură valorea din acel set de date. Dacă avem 1000 de valori, o să se antreneze pe 999 și validarea se va face pe o valoare. Se va repeta acest proces până când se validează pe fiecare valoare din setul de date. scoring face referire la ce metrică să folosească pentru partea de validare. Se merge pe ideea de valoare mai mare reprezintă un model mai bun, de aceea pentru metricii de mean_square_error și mean_absolute_error se va returna valoarea negatică (neg_mean_sqaured_error, neg_mean_absolute_error)

In [16]:
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)

In [17]:
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

După cum spuneam, ceea ce se retunrează este valoarea negativă de la mean_squared_error, de aceea valorile sunt cu minus. Putem să transformăm aceste valori în valori pozitive.

In [18]:
abs(scores)

array([ 9.32552967,  4.9449624 , 11.39665242,  7.0242106 ,  8.38562723])

Spuneam că valoarea erorii care trebuie considerată corectă este valoarea medie pentru fiecare valiadre în parte, de aceea pentru array-ul scores trebuie să accesăm media (scores.mean()). Pentru a fi o valoarea pozitivă o să ne folosim din nou de metoda abs()

In [19]:
abs(scores.mean())

8.215396464543607

Valoarea medie a erorii este 8.21, ceea ce nu este bine. Putem să modificăm valoarea pentru hyperparametrul alpha. În continuare o să modificăm valoarea să fie 1 și o să realizăm din nou partea de cross-validation. Pentru asta o să creem un nou model (model=Ridge(alpha=1)) și se va rerula codul de cross-validation

In [20]:
model2 = Ridge(alpha=1)

In [22]:
scores2 = cross_val_score(model2, X_train, y_train, scoring='neg_mean_squared_error', cv=5)

In [23]:
abs(scores2.mean())

3.344839296530695

După ce am modificat valoarea pentru aplha la 1 calitatea modelului a crescut. Ce este interesant la această metodă este faptul că nu returnează un model care este antrenat (pe care s-a apelat metoda .fit()). Pentru a crea un model antrenat și a face predicții trebuie să antrenăm acel model și să facem predicțiile

In [25]:
model2.fit(X_train, y_train)

Ridge(alpha=1)

In [26]:
y_pred = model2.predict(X_test)

In [27]:
from sklearn.metrics import mean_squared_error

In [28]:
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))

In [29]:
RMSE

1.5228334050147283

## Recapitulare

În cadrul acestei lecții am învățat:

    1. De unde să importăm metoda cross_val_score

        from sklearn.model_selection import cross_val_score

    2. De ce parametrii are nevoie metoda pentru a funcționa

        estimator = modelul pe care dorim să îl folosim

        X = setul de date de antrenare pentru features

        y = setul de date de antrenare pentru labels

        scoring = metrica pe care să o folosească pentru evaluarea modelului

        cv = numărul de K-folds în care să împartă setul de date de antrenare

    3. Cum să utilizăm metoda cross_val_score

        scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)

    4. Cum să aflăm media erorilor modelului

        abs(scores.mean())

    5. Modelul nu este antrenat, pentru a face predicții finale la final de cross-validation trebuie să antrenăm modelul și să facem predicții pentru setul de date de testare

        model.fit(X_train)

        y_pred = model.predict(X_test)